In [1]:
import numpy as np
import pandas as pd
import re, json, csv, requests, time, glob, tqdm
from bs4 import BeautifulSoup
from selenium import webdriver
from collections import Counter

In [2]:
def remove_rt(text):
    return re.sub('<rt>.+?</rt>', '', text)

def tag(text):
    text = re.sub(r'<span class="colorC">(.+?)</span>', r"{org}\1{/org}", text)
    text = re.sub(r'<span class="colorL">(.+?)</span>', r"{plc}\1{/plc}", text)
    text = re.sub(r'<span class="colorN">(.+?)</span>', r"{per}\1{/per}", text)
    return text

def retag(text):
    text = re.sub(r'{org}(.+?){/org}', r"<org>\1</org>", text)
    text = re.sub(r'{plc}(.+?){/plc}', r"<plc>\1</plc>", text)
    text = re.sub(r'{per}(.+?){/per}', r"<per>\1</per>", text)
    return text

def remove_a(text):
    text = re.sub(r'</?a.*?>', '', text)
    text = re.sub(r'<span class="under">(\w+)</span>', r'\1', text)
    text = re.sub(r'<img.+?>(<br ?/?>)?', '', text)
    text = re.sub(r'^<br ?/?>', '', text)
    return text.strip()

# for old web easy
def scrape_easy_one(html):
    soup = BeautifulSoup(html, "html.parser")
    url_normal = soup.find('div', id="regularnews").a.get('href')
    if '/http://' in url_normal:
        url_normal = 'http://' + url_normal.split('/http://')[-1]
    else:
        url_normal = 'https://' + url_normal.split('/https://')[-1]
    date = soup.find('p', id="newsDate").text[1:-1]
    #title_easy = soup.find('h1', class_="article-main__title")
    #title_easy_ruby = ''.join([str(t) for t in title_easy.contents]).strip()
    url_easy = soup.find('meta', attrs={'name':'shorturl'}).get('content')
    title_easy = soup.find('div', id='newstitle').h2
    title_easy_ruby = ''.join([str(t) for t in title_easy.contents]).strip()
    title_easy = BeautifulSoup(remove_rt(str(title_easy)), "html.parser").text.strip()
    article_easy = soup.find('div', id="newsarticle")
    article_easy = BeautifulSoup(tag(remove_rt(str(article_easy))), "html.parser").text.strip()
    article_easy_ruby = soup.find('div', id="newsarticle").find_all('p')
    article_easy_ruby = '\n'.join([''.join([remove_a(str(l)) for l in p.contents]) for p in article_easy_ruby if p != []]).strip()
    
    return {
        'id':url_easy.split('/')[-1].split('.html')[0],
        'title_easy':title_easy,
        'title_easy_ruby':title_easy_ruby,
        'article_easy':retag(article_easy),
        'article_easy_ruby':article_easy_ruby,
        'url_easy':url_easy,
        'url_normal':url_normal,
        'date_easy':date
    }

# for new web easy
def scrape_easy_one_new(html):
    soup = BeautifulSoup(html, "html.parser")
    url_easy = 'https://' + soup.find('meta', property="og:url").get('content').split('/https://')[-1]
    url_normal = soup.find('div', class_="link-to-normal").a.get('href')
    date = soup.find('p', class_="article-main__date").text[1:-1]
    title_easy = soup.find('h1', class_="article-main__title")
    title_easy_ruby = ''.join([str(t) for t in title_easy.contents]).strip()
    title_easy = BeautifulSoup(remove_rt(str(title_easy)), "html.parser").text.strip()
    article_easy = soup.find('div', class_="article-main__body article-body")
    article_easy = BeautifulSoup(tag(remove_rt(str(article_easy))), "html.parser").text.strip()
    article_easy_ruby = soup.find('div', class_="article-main__body article-body").find_all('p')
    article_easy_ruby = '\n'.join([''.join([remove_a(str(l)) for l in p.contents]) for p in article_easy_ruby if p != []]).strip()
    
    return {
        'id':url_easy.split('/')[-1].split('.html')[0],
        'title_easy':title_easy,
        'title_easy_ruby':title_easy_ruby,
        'article_easy':retag(article_easy),
        'article_easy_ruby':article_easy_ruby,
        'url_easy':url_easy,
        'url_normal':url_normal,
        'date_easy':date
    }



def get_link(start=0):
    notyet = []
    n_list = pd.read_json('nhkweb.json', encoding='utf-8')['url'].tolist()
    df_e = pd.read_json('nhkwebeasy.json', encoding='utf-8') 
    for i in df_e['url_normal'][start:]:
        if i not in n_list:
            notyet.append(i)
    with open('nolinknormal.txt') as f:
        nolink = f.read().split()
    return sorted(set(notyet) - set(nolink))
        
def js_e(dic):
    with open('nhkwebeasy.json', 'r', encoding='utf-8') as f:
        data = json.load(f)
    with open('nhkwebeasy.json', 'w', encoding='utf-8') as f:
        if dic['id'] not in [x['id'] for x in data]:
            data.append(dic)
        else:
            for i, d in enumerate(data):
                if dic['id'] == d['id']:
                    data[i] = dic
        data = sorted(data, key=lambda x:x['id'])
        json.dump(data, f, indent=4, ensure_ascii=False)

# scrape

In [4]:
driver = webdriver.Chrome()

# clean category & keyword

In [29]:
with open('nhkweb.json','r', encoding='utf-8') as f:
    data = json.load(f)
    print(len(data))
with open('nhkwebeasy.json','r', encoding='utf-8') as f:
    data = json.load(f)
    print(len(data))

7979
4744


In [15]:
# check category

with open('nhkweb.json','r', encoding='utf-8') as f:
    data = json.load(f)
print('articles: ', len(data))
genre = Counter()
for dic in data:
    for g in dic['genre']:
        genre[g] += 1
genre.most_common()

articles:  7821


[('社会', 2375),
 ('国際', 2079),
 ('科学・文化', 1268),
 ('ビジネス', 1195),
 ('スポーツ', 980),
 ('政治', 739),
 ('暮らし', 606),
 ('地域', 512),
 ('気象・災害', 354)]

In [14]:
# genre <> keywords

with open('nhkweb.json','r', encoding='utf-8') as f:
    data = json.load(f)

category = ['社会', '国際', 'ビジネス', 'スポーツ', '政治', '科学・文化', '暮らし', '地域', '気象・災害']
for i, dic in enumerate(data):
    newgenre = []
    newkey = []
    for j in dic['genre']:
        if j in category:
            newgenre.append(j)
        elif j == "災害" or j == "気象":
            newgenre.append('気象・災害')
        elif j == "科学・医療" or j == "文化・エンタメ":
            newgenre.append('科学・文化')
        elif j == "経済":
            newgenre.append('ビジネス')
        else:
            newkey.append(j)
    for j in dic['keywords']:
        if j in category:
            newgenre.append(j)
        elif j == "災害" or j == "気象":
            newgenre.append('気象・災害')
        else:
            newkey.append(j)
    data[i]['genre'] = list(set(newgenre))
    data[i]['keywords'] = list(set(newkey))

with open('nhkweb.json','w', encoding='utf-8') as f:
    json.dump(data, f, indent=4, ensure_ascii=False)
    

In [3]:
normal = pd.read_json('nhkwebeasy.json')
normal.id.value_counts()

k10010353911000    1
k10010441541000    1
k10012115201000    1
k10010055061000    1
k10010471031000    1
                  ..
k10010771931000    1
k10010399941000    1
k10010904951000    1
k10011871991000    1
k10011099221000    1
Name: id, Length: 4530, dtype: int64

# NHK web easy (new) ID k1001140020 ~

In [21]:
with open('tobescraped.txt') as f:
    urls = f.read().split()
print(len(urls))

ids = pd.read_json('nhkwebeasy.json')['id'].tolist()
urls = [url for url in urls if url.split('/')[-1].strip('.html') not in ids]
print(len(urls))
del ids

1841
3


In [4]:
driver = webdriver.Chrome()

In [22]:
urls

['https://web.archive.org/web/*/http://www3.nhk.or.jp:80/news/easy/k10010731741000/k10010731741000.html',
 'https://web.archive.org/web/*/http://www3.nhk.or.jp:80/news/easy/k10010729201000/k10010729201000.html',
 'https://web.archive.org/web/*/http://www3.nhk.or.jp:80/news/easy/k10010705131000/k10010705131000.html']

In [15]:
for url in urls:
    driver.get(url)
    time.sleep(6)
    html = str(driver.page_source.encode('utf-8'))
    snap = re.search(r'(between|1 time|Saved \d+ times).*?<a href="(.+?)">', html)
    if snap == None:
        continue
    driver.get('https://web.archive.org' + snap.group(2))
    time.sleep(10)
    html = driver.page_source.encode('utf-8')
    try:
        dic = scrape_easy_one(html)
    except:
        dic = scrape_easy_one_new(html)
    js_e(dic)

In [27]:
html = driver.page_source.encode('utf-8')
dic = scrape_easy_one(html)
dic

{'id': 'k10010705131000',
 'title_easy': 'シリア\u3000戦争をやめる約束を守らない',
 'title_easy_ruby': 'シリア\u3000<ruby>戦争<rt>せんそう</rt></ruby>をやめる<ruby>約束<rt>やくそく</rt></ruby>を<ruby>守<rt>まも</rt></ruby>らない',
 'article_easy': '<plc>シリア</plc>は国の中で、<per>アサド</per>大統領の政府と、政府に反対する人たちが戦争をしています。<plc>アメリカ</plc>と<plc>ロシア</plc>が間に入って、それぞれの人たちは１２日夜から戦争をやめる約束をしました。\nしかし、１９日に<plc>シリア</plc>の政府が攻撃を始めました。政府は、政府に反対している人たちを助けている<plc>アメリカ</plc>などが攻撃したからやったと言いました。\nそのあと、<plc>シリア</plc>で薬などを運んでいた車が攻撃されて、国連の人などが大勢亡くなりました。<plc>アメリカ</plc>は、<plc>シリア</plc>の政府か政府を助けている<plc>ロシア</plc>が攻撃したと言いました。\n今<plc>シリア</plc>では空からの攻撃などが続いていて、大勢の人が亡くなっています。\n２５日の国連の会議では、<plc>シリア</plc>での戦争について<plc>アメリカ</plc>と<plc>ロシア</plc>が自分たちの意見を言って争いました。戦争をやめるという約束は守らないまま、なくなってしまうかもしれません。',
 'article_easy_ruby': '<span class="colorL">シリア</span>は<ruby>国<rt>くに</rt></ruby>の<ruby>中<rt>なか</rt></ruby>で、<span class="colorN">アサド</span><ruby>大統領<rt>だいとうりょう</rt></ruby>の<ruby>政府<rt>せいふ</rt></ruby>と、<ruby>政府<rt>せいふ</rt></ruby>に<ruby>反対<rt>はんたい</rt></ruby>する<ruby>人<rt>ひと</rt>

In [28]:
js_e(dic)

In [36]:
soup = BeautifulSoup(html)
soup.find('div', class_='link-to-normal')

<div class="link-to-normal" id="js-regular-news-wrapper">
<a class="btn" href="https://www3.nhk.or.jp/news/html/20200206/k10012274671000.html" id="js-regular-news" target="_blank"><ruby>普通<rt>ふつう</rt></ruby>のニュースを<ruby>読<rt>よ</rt></ruby>む</a>
</div>